In [4]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    data = data.drop(columns=['ID'])  # Dropping the 'ID' column
    # Normalize the data
    data = data.dropna()
    for col in data.columns:
        data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
    return data

In [6]:
filepath = "/kaggle/input/xxxxxxxxxxxxxxxxxxx/custprofile.csv"
df = load_and_preprocess_data(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Income               2216 non-null   float64
 1   Kidhome              2216 non-null   float64
 2   Teenhome             2216 non-null   float64
 3   Recency              2216 non-null   float64
 4   MntWines             2216 non-null   float64
 5   MntFruits            2216 non-null   float64
 6   MntMeatProducts      2216 non-null   float64
 7   MntFishProducts      2216 non-null   float64
 8   MntSweetProducts     2216 non-null   float64
 9   MntGoldProds         2216 non-null   float64
 10  NumDealsPurchases    2216 non-null   float64
 11  NumWebPurchases      2216 non-null   float64
 12  NumCatalogPurchases  2216 non-null   float64
 13  NumStorePurchases    2216 non-null   float64
 14  NumWebVisitsMonth    2216 non-null   float64
dtypes: float64(15)
memory usage: 277.0 KB


In [7]:
def k_means(data, k, iterations):
    if isinstance(data, pd.DataFrame):
        data = data.to_numpy()
    np.random.seed(42)  # Ensure reproducibility
    
    # Initialize centroids randomly
    initial_indices = np.random.choice(data.shape[0], size=k, replace=False)
    centroids = data[initial_indices]

    for _ in range(iterations):
        clusters = defaultdict(list)

        # Assign points to the nearest centroid
        for index, point in enumerate(data):
            distances = [1 - cosine_similarity([point], [centroid])[0][0] for centroid in centroids]
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(index)

        # Update centroids
        new_centroids = []
        for cluster_indices in clusters.values():
            cluster_data = data[cluster_indices]
            new_centroids.append(np.mean(cluster_data, axis=0))
        centroids = np.array(new_centroids)

    return clusters


In [8]:
def silhouette_score(data, clusters):
    data_np = np.array(data)
    a_values, b_values = [], []
    
    cosine_distances = 1 - cosine_similarity(data_np)
    
    for index, point in enumerate(data_np):
        for cluster_index, indices in clusters.items():
            if index in indices:
                current_cluster_indices = indices
                break
        
        a = np.mean([cosine_distances[index, i] for i in current_cluster_indices if i != index]) if len(current_cluster_indices) > 1 else 0
        b = np.inf
        for other_cluster_index, other_indices in clusters.items():
            if other_cluster_index != cluster_index:
                dist = np.mean([cosine_distances[index, i] for i in other_indices])
                b = min(b, dist)
                
        a_values.append(a)
        b_values.append(b)
    
    s_scores = [(b - a) / max(a, b) if max(a, b) > 0 else 0 for a, b in zip(a_values, b_values)]
    return np.mean(s_scores)

In [9]:
# def k_means(data, k, iterations):
#     # Ensure input is a NumPy array
#     data_np = np.array(data) if not isinstance(data, np.ndarray) else data
    
#     # Randomly initialize centroids
#     np.random.seed(42)
#     initial_indices = np.random.choice(len(data_np), size=k, replace=False)
#     centroids = data_np[initial_indices]
    
#     for _ in range(iterations):
#         clusters = defaultdict(list)

#         # Assign data points to the nearest cluster
#         for index, point in enumerate(data_np):
#             # Use distance as 1 - cosine similarity for clustering
#             distances = [1 - cosine_similarity([point], [centroid])[0][0] for centroid in centroids]

#             cluster_index = np.argmin(distances)
#             clusters[cluster_index].append(index)
        
#         # Update centroids
#         new_centroids = []
#         for cluster_indices in clusters.values():
#             if cluster_indices:  # Ensure the cluster is not empty
#                 cluster_data = data_np[cluster_indices]
#                 new_centroids.append(np.mean(cluster_data, axis=0))
#             else:  # Handle the rare case of an empty cluster
#                 new_centroids.append(centroids[len(new_centroids)])  # Reuse the old centroid
#         centroids = new_centroids
    
#     # Save clustering information
    
#     return clusters

# def silhouette_score(data, clusters):
#     data_np = np.array(data) if not isinstance(data, np.ndarray) else data
#     a_values, b_values = [], []
    
#     # Precompute all pairwise cosine distances for efficiency
#     cosine_distances = 1 - cosine_similarity(data_np)
    
#     for index, point in enumerate(data_np):
#         # Identify the cluster of the current point
#         for cluster_index, indices in clusters.items():
#             if index in indices:
#                 current_cluster_indices = indices
#                 break
        
#         # Compute 'a' value
#         if len(current_cluster_indices) > 1:
#             a = np.mean([cosine_distances[index, i] for i in current_cluster_indices if i != index])
#         else:
#             a = 0
        
#         # Compute 'b' value: smallest mean distance to all points in any other cluster
#         b = np.inf
#         for other_cluster_index, other_indices in clusters.items():
#             if other_cluster_index != cluster_index:
#                 dist = np.mean([cosine_distances[index, i] for i in other_indices])
#                 b = min(b, dist)
        
#         a_values.append(a)
#         b_values.append(b)
    
#     # Calculate silhouette scores
#     s_scores = [(b - a) / max(a, b) if max(a, b) > 0 else 0 for a, b in zip(a_values, b_values)]
    
#     # Return the mean silhouette score
#     return np.mean(s_scores)

In [10]:
best_k = 3
best_score = -1
best_clusters = defaultdict(list)
for k in range(3, 7):  # Exploring k values from 3 to 6
    clusters = k_means(df, k, 20)
#     print(clusters)
    score = silhouette_score(df, clusters)
    print(f"Silhouette Score for k={k}: {score}")
    if score > best_score:
        best_k, best_score = k, score
        best_clusters = clusters
print(f"Best k={best_k}")
print(f"Best Score for k={best_k}: {best_score}")



Silhouette Score for k=3: 0.40240389264387794
Silhouette Score for k=4: 0.3626070522240497
Silhouette Score for k=5: 0.36889402279492517
Silhouette Score for k=6: 0.3344620014988835
Best k=3
Best Score for k=3: 0.40240389264387794


In [11]:

class CustomTopDownClustering:
    def __init__(self, min_clusters=3):
        # Initialize the minimum number of clusters
        self.min_clusters = min_clusters
        # Store the indices of data points in each cluster
        self.cluster_indices = []

    def fit(self, X):
        # Initialize with one cluster containing all data points
        self.cluster_indices = [list(range(len(X)))]

        # Continue splitting until reaching the desired number of clusters
        while len(self.cluster_indices) < self.min_clusters:
            # Find the cluster with the highest average cosine distance
            max_avg_cos_distance = -1
            split_cluster_index = None

            # Iterate over existing clusters to find the one to split
            for i, cluster_index in enumerate(self.cluster_indices):
                # Calculate the average cosine distance within the cluster
                avg_cos_distance = self._average_cosine_distance(X, cluster_index)
                # Update if the current cluster has higher average cosine distance
                if avg_cos_distance > max_avg_cos_distance:
                    max_avg_cos_distance = avg_cos_distance
                    split_cluster_index = i

            # Split the cluster along the dimension with the highest average cosine distance
            split_cluster_indices = self.cluster_indices[split_cluster_index]

            # Perform the split
            left_indices, right_indices = self._split_data(X, split_cluster_indices)

            # Update the cluster list with the two split clusters
            self.cluster_indices.pop(split_cluster_index)
            self.cluster_indices.append(left_indices)
            self.cluster_indices.append(right_indices)
            
        return self.cluster_indices

    def cosine_distance(self, a, b):
        # Compute the cosine distance between two vectors
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        return 1 - (dot_product / (norm_a * norm_b))

    def _average_cosine_distance(self, X, cluster_indices):
        # Calculate the average cosine distance within a cluster
        distances = []
        for i in range(len(cluster_indices)):
            for j in range(i + 1, len(cluster_indices)):
                distances.append(self.cosine_distance(X[cluster_indices[i]], X[cluster_indices[j]]))
        return np.mean(distances)
    
    def _split_data(self, X, cluster_indices):
        # Initialize cluster centroids randomly
        while True:
            centroid1_idx, centroid2_idx = np.random.choice(len(cluster_indices), size=2, replace=False)
            if centroid1_idx != centroid2_idx:
                break
        centroid1 = X[cluster_indices[centroid1_idx]]
        centroid2 = X[cluster_indices[centroid2_idx]]

        # Perform K-means clustering to split the data
        for _ in range(100):  # Number of iterations for K-means
            cluster1_indices = []
            cluster2_indices = []
            for idx in cluster_indices:
                # Assign each point to the nearest centroid
                dist_to_centroid1 = self.cosine_distance(X[idx], centroid1)
                dist_to_centroid2 = self.cosine_distance(X[idx], centroid2)
                if dist_to_centroid1 < dist_to_centroid2:
                    cluster1_indices.append(idx)
                else:
                    cluster2_indices.append(idx)

            # Update centroids based on the mean of points in each cluster
            centroid1 = np.mean(X[cluster1_indices], axis=0)
            centroid2 = np.mean(X[cluster2_indices], axis=0)

        return cluster1_indices, cluster2_indices


In [12]:
model= CustomTopDownClustering(min_clusters=best_k)
X=df.values
heir_cluster_points = model.fit(X)

for i in range (len(heir_cluster_points)):
    heir_cluster_points[i]=np.array(heir_cluster_points[i])
    
    
for i in range(len(heir_cluster_points)):
    heir_cluster_points[i] = np.sort(heir_cluster_points[i])

# Sort the list of arrays based on the first element of each array
heir_cluster_points.sort(key=lambda y: y[0])



In [13]:
best_cluster_points=[]
for i in range(len(best_clusters)):
    best_cluster_points.append(best_clusters[i])
for i in range (len(best_cluster_points)):
    best_cluster_points[i]=np.array(best_cluster_points[i])
#     print(best_cluster_points[i])
# print(best_cluster_points)    
    
for i in range(len(best_cluster_points)):
    best_cluster_points[i] = np.sort(best_cluster_points[i])

# Sort the list of arrays based on the first element of each array
best_cluster_points.sort(key=lambda y: y[0])
with open("kmeans.txt", "w") as f:
    f.write("Number of clusters = {best_score}\n")
    for row in best_cluster_points:
        f.write(', '.join(map(str, row)))
        f.write('\n')
        
with open("divisive.txt", "w") as f:
    f.write("Number of clusters = {}\n".format(best_k))
    for row in heir_cluster_points:
        f.write(', '.join(map(str, row)))
        f.write('\n')

In [14]:
import numpy as np

def jaccard_similarity(arrA, arrB):
    # Calculate the Jaccard similarity between two arrays
    intersection = len(np.intersect1d(arrA, arrB))
    union = len(np.union1d(arrA, arrB))
    return intersection / union if union != 0 else 0

def mapping_and_similarity(list1, list2):
    # Function to compute similarity scores and mappings between two lists of arrays
    similarity_scores = []
    
    # Iterate over each array in list1
    for arrA in list1:
        max_similarity = 0
        best_mapping = None
        
        # Iterate over each array in list2
        for arrB in list2:
            # Check if arrB has been used in previous mappings
            if any(np.array_equal(arrB, mapping[1]) for mapping in similarity_scores):
                continue
            
            # Compute Jaccard similarity for current mapping
            similarity = jaccard_similarity(arrA, arrB)
            
            # Update max similarity and corresponding mapping
            if similarity > max_similarity:
                max_similarity = similarity
                best_mapping = (arrA, arrB)
        
        # Store the max similarity and corresponding mapping for the current array in list1
        similarity_scores.append((max_similarity, best_mapping))
    
    return similarity_scores

# Example usage
# Compute similarity scores between heir_cluster_points and best_cluster_points
similarity_scores = mapping_and_similarity(heir_cluster_points, best_cluster_points)

# Print the results
for i, (similarity, mapping) in enumerate(similarity_scores):
    print(f"Jaccard Similarity for pair {i+1}: {similarity}")
    # Additional details can be printed if needed
    # print("For cluster of divisive\n",mapping[0])
    # print("Mapping to kmeans cluster\n", mapping[1])
    print()

# Compute average Jaccard Similarity
average_similarity=0.0
for (similarity,mapping) in (similarity_scores):
    average_similarity+=similarity

average_similarity/=best_k
print(f"Average Jaccard Similarity: {average_similarity}\n")


Jaccard Similarity for pair 1: 0.9161490683229814

Jaccard Similarity for pair 2: 0.8137869292748433

Jaccard Similarity for pair 3: 0.6368715083798883

Average Jaccard Similarity: 0.7889358353259043

